# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [50]:
import requests
import re
import math
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [243]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')

# 尋找正在上映中的全部電影筆數，其所在的tag
html = soup.find("div", attrs={'class':'release_box'})  
total = re.findall(r'\d+', html.p.string) 
print("正在上映中總共: ", total[0],'部電影\n')

# 總頁數
max_page = math.ceil(int(total[0]) / 10)

# 接下來要爬取每一頁的電影介紹
# 先點選網頁最底部的下一頁，觀察其網址變化。然後我們把頁數當作變數傳入，用迴圈爬取。

for page_number in range(1, max_page+1, 1):
    url = 'https://movies.yahoo.com.tw/movie_intheaters.html?page='+ str(page_number)
    resp = requests.get(url)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, 'lxml')

    for d in soup.find_all(class_="release_movie_name"):
        movieInfo = re.findall(r'\d+',d.find('a')['href'])
        print('電影代號:%5d' % int(movieInfo[-1]),'  片名: %s' % d.find('a').text.replace(' ', '').replace('\n', ''))




正在上映中總共:  78 部電影

電影代號:10256   片名: 去年聖誕節
電影代號:10264   片名: 呆萌特務
電影代號:10279   片名: 抓狂博物館
電影代號:10287   片名: 人間失格：太宰治與他的3個女人
電影代號:10328   片名: 絕世情歌
電影代號:10336   片名: 我的靈魂是愛做的
電影代號:10339   片名: 女孩要革命
電影代號:10340   片名: 花生醬獵鷹的願望
電影代號:10352   片名: 一屍到底番外篇：好萊塢大作戰！
電影代號:10364   片名: 特約經紀公司
電影代號:10134   片名: 野蠻遊戲：全面晉級
電影代號:10044   片名: 暴走曼哈頓
電影代號:10224   片名: 布魯克林孤兒
電影代號:10254   片名: 懸案密碼前傳：瓶中信
電影代號:10282   片名: 寂寞診療室
電影代號:10327   片名: 夕霧花園
電影代號:10097   片名: 賽道狂人
電影代號:10258   片名: 鋒迴路轉
電影代號:10271   片名: BanGDream！FILMLIVE
電影代號:10298   片名: 82年生的金智英
電影代號:10358   片名: 麂皮：永不滿足
電影代號: 9597   片名: 冰雪奇緣2
電影代號:10226   片名: 為美好的世界獻上祝福！紅傳說
電影代號:10260   片名: 神機有毛病
電影代號:10281   片名: 萬萬沒想到
電影代號:10296   片名: 大餓
電影代號:10302   片名: HelloWorld
電影代號:10098   片名: 安眠醫生
電影代號:10130   片名: 決戰中途島
電影代號:10290   片名: 重甲機神：神降臨
電影代號:10301   片名: 山田孝之的痛苦與榮耀
電影代號:10067   片名: 魔鬼終結者：黑暗宿命
電影代號: 9992   片名: 阿達一族
電影代號:10240   片名: 陽光普照
電影代號:10292   片名: 紫羅蘭永恆花園外傳－永遠與自動手記人偶－
電影代號:10284   片名: 狙擊獵殺：救援行動
電影代號:10017   片名: 雙子殺手
電影代號:10267   片名: 盲人律師
電影代號:10262   片名: 黑魔女

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [277]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10264

In [278]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 台中, 代號(area_id): 2
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17
放映地區: 金門, 代號(area_id): 24
放映地區: 澎湖, 代號(area_id): 23


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [279]:
# 指定放映地區
area_id = 28

In [280]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

十二月 15
十二月 16
十二月 17
十二月 18
十二月 19


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [284]:
# 選定要觀看的日期
date = "2019-12-16"

In [364]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [365]:
for data in html:
  print('放映電影院:',data.a.text)
  movietypeInfo = data.find("li", attrs={'class':'taps'}).find_all('span')
  movie_type = ''

  for singleType in movietypeInfo:
    movie_type += singleType.text + ' '

  print('放映類型:', movie_type)

  movieTimeInfo = data.find("div", attrs={'class':'input_picker jq_input_picker'}).find_all('label')
  movie_time = ''

  for singleTime in movieTimeInfo:
    movie_time += date +' '+ singleTime.text + '\n'

  print('放映時間:\n'+movie_time)


放映電影院: 台北信義威秀影城
放映類型: 數位 
放映時間:
2019-12-16 11:50
2019-12-16 17:30

放映電影院: 喜樂時代影城南港店
放映類型: 數位 中文 
放映時間:
2019-12-16 10:20
2019-12-16 13:50
2019-12-16 15:35
2019-12-16 17:20

